**Home Exercise:**

a) Improve the model by using interpolation smoothing with the "Stupid Backoff" method (Brants et al., 2007).

b) Compare with the results from In Class Exercise.

c) Use the newly built model to generate the next words for a given word sequence.

d) Combine with a function that calculates the distance between words to predict the correct word for a misspelled word position. (from difflib import get_close_matches)

# Import

In [ ]:
%pip install gdown matplotlib


  Using cached gdown-5.2.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached soupsieve-2.6-py3-none-any.whl.metadata (4.6 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
Using cached gdown-5.2.0-py3-none-any.whl (18 kB)
Using cached PySocks-1.7.1-py3-none-any.whl (16 kB)
Using cached soupsieve-2.6-py3-none-any.whl (36 kB)



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# import sys
# import os
# import platform

# # Python environment details
# print("Python executable being used:", sys.executable)
# print("Python version:", sys.version)

# # Operating System details
# print("Operating System:", platform.system())
# print("OS Version:", platform.version())
# print("OS Release:", platform.release())

# # Machine and architecture details
# print("Machine:", platform.machine())

# # Visual Studio Code details (based on environment variable)
# vscode_info = os.environ.get('VSCODE_PID', None)
# if vscode_info:
#     print("Running in Visual Studio Code")
# else:
#     print("Not running in Visual Studio Code")

Python executable being used: c:\Users\User\AppData\Local\Programs\Python\Python311\python.exe
Python version: 3.11.5 (tags/v3.11.5:cce6ba9, Aug 24 2023, 14:38:34) [MSC v.1936 64 bit (AMD64)]
Operating System: Windows
OS Version: 10.0.19045
OS Release: 10
Machine: AMD64
Running in Visual Studio Code


In [3]:
import os
from collections import defaultdict
import math
import gdown
import re
import matplotlib.pyplot as plt
from difflib import get_close_matches

# Function Definitions

## Load data

In [4]:
def load_data(filepath):
    """
    Load and preprocess text data from a file.
    """
    with open(filepath, "r", encoding="utf-8") as file:
        sentences = file.readlines()
    
    # Add <s> and </s> to each sentence and tokenize
    tokens = []
    for sentence in sentences:
        sentence = sentence.strip().lower()
        if sentence:  
            tokens.extend(["<s>"] + sentence.split() + ["</s>"])
    
    return tokens

## Build n-gram model

In [5]:
def build_ngram_model(corpus, n):
    """
    Build an n-gram model from a given corpus.
    """
    ngram_counts = defaultdict(int)
    n_minus_1_counts = defaultdict(int)
    vocabulary = set(corpus)

    for i in range(len(corpus) - n + 1):
        ngram = tuple(corpus[i:i + n])
        n_minus_1_gram = tuple(corpus[i:i + n - 1])
        ngram_counts[ngram] += 1
        n_minus_1_counts[n_minus_1_gram] += 1

    return ngram_counts, n_minus_1_counts, len(vocabulary)

## Compute Sentence Probabilities with laplace smoothed probabilities

In [6]:
def compute_laplace_probability(ngram, ngram_counts, n_minus_1_counts, vocab_size):
    """
    Compute the Laplace smoothed probability of an n-gram.
    """
    ngram_count = ngram_counts[ngram]
    n_minus_1_count = n_minus_1_counts[ngram[:-1]] if len(ngram) > 1 else sum(ngram_counts.values())
    return (ngram_count + 1) / (n_minus_1_count + vocab_size)

In [7]:
def sentence_probability(sentence, ngram_counts, n_minus_1_counts, vocab_size, n):
    """
    Compute the probability of a sentence using an n-gram model with laplace smoothing.
    """
    tokens = ["<s>"] + sentence.split() + ["</s>"]
    prob = 1.0
    for i in range(len(tokens) - n + 1):
        ngram = tuple(tokens[i:i + n])
        prob *= compute_laplace_probability(ngram, ngram_counts, n_minus_1_counts, vocab_size)
    return prob

## Compute Sentence Probabilities with Stupid Backoff method

In [8]:
def compute_stupid_backoff_probability(ngram, ngram_models, alpha=0.4, laplace_smoothing=True):
    """
    Compute probability using Stupid Backoff method combined with optional Laplace smoothing.

    Returns:
        float: The probability or relative score of the n-gram.
    """
    for n in range(len(ngram), 0, -1):  # Start with highest n-gram and backoff to lower-order
        ngram_model = ngram_models.get(n)
        if ngram_model:
            ngram_counts, n_minus_1_counts, vocab_size = ngram_model
            
            if ngram in ngram_counts:
                ngram_count = ngram_counts[ngram]
                n_minus_1_count = n_minus_1_counts[ngram[:-1]] if len(ngram) > 1 else sum(ngram_counts.values())

                if laplace_smoothing:
                    # Laplace smoothing ensures non-zero probability
                    return (ngram_count + 1) / (n_minus_1_count + vocab_size)
                else:
                    # Stupid Backoff with relative frequency
                    return ngram_count / n_minus_1_count if n_minus_1_count > 0 else 0

    # Backoff to unigram model if higher-order n-grams are not found
    unigram_model = ngram_models.get(1)
    if unigram_model:
        unigram_counts, _, vocab_size = unigram_model
        unigram_count = unigram_counts.get((ngram[-1],), 0)  # Frequency of the unigram
        total_count = sum(unigram_counts.values())  # Total number of tokens

        if laplace_smoothing:
            # Laplace smoothing for unigrams
            return (unigram_count + 1) / (total_count + vocab_size)
        else:
            # Stupid Backoff with relative frequency
            return unigram_count / total_count if total_count > 0 else 0

    # Fallback: Return the alpha (backoff factor) for unseen cases
    return alpha


In [9]:
def sentence_probability_with_backoff(sentence, ngram_models, alpha=0.4):
    """
    Compute the probability of a sentence using Stupid Backoff smoothing.
    """
    tokens = ["<s>"] + sentence.split() + ["</s>"]
    prob = 1.0
    for i in range(len(tokens)):
        for n in range(3, 0, -1):  # Try 3-gram, 2-gram, then 1-gram
            ngram = tuple(tokens[i:i + n])
            if len(ngram) == n:
                prob *= compute_stupid_backoff_probability(ngram, ngram_models, alpha)
                break
    return prob


## Compare sentences by using soothing with laplace and with stupid backoff

In [10]:
def compare_sentences_with_smoothing(sentence1, sentence2, ngram_models, alpha=0.4):
    """
    Compare probabilities and perplexities of two sentences using both Laplace and Stupid Backoff.
    """
    results = {"Laplace": {}, "Stupid Backoff": {}}

    # Compare using Laplace smoothing
    for n in range(1, 4):
        ngram_model = ngram_models[n]
        
        sentence1_prob_laplace = sentence_probability(sentence1, *ngram_model, n)
        sentence2_prob_laplace = sentence_probability(sentence2, *ngram_model, n)
        
        if sentence1_prob_laplace > sentence2_prob_laplace:
            higher_sentence = "Sentence 1 (Correct)"
        elif sentence2_prob_laplace > sentence1_prob_laplace:
            higher_sentence = "Sentence 2 (Incorrect)"
        else:
            higher_sentence = "Both sentences have equal probability"
        
        results["Laplace"][f"{n}-gram"] = {
            "sentence1": sentence1_prob_laplace,
            "sentence2": sentence2_prob_laplace,
            "higher_probability": higher_sentence,
            "probability_difference": abs(sentence1_prob_laplace - sentence2_prob_laplace),
        }

    # Compare using Stupid Backoff smoothing
    sentence1_prob_backoff = sentence_probability_with_backoff(sentence1, ngram_models, alpha)
    sentence2_prob_backoff = sentence_probability_with_backoff(sentence2, ngram_models, alpha)

    if sentence1_prob_backoff > sentence2_prob_backoff:
        higher_sentence = "Sentence 1 (Correct)"
    elif sentence2_prob_backoff > sentence1_prob_backoff:
        higher_sentence = "Sentence 2 (Incorrect)"
    else:
        higher_sentence = "Both sentences have equal probability"

    results["Stupid Backoff"] = {
        "sentence1": sentence1_prob_backoff,
        "sentence2": sentence2_prob_backoff,
        "higher_probability": higher_sentence,
        "probability_difference": abs(sentence1_prob_backoff - sentence2_prob_backoff),
    }

    return results


## Function for predicting the next top-k tokens

The Stupid Backoff method scores n-grams using relative frequencies but does **not normalize** probabilities to sum to 1. It uses a backoff mechanism to handle cases where an n-gram does not exist in the training data.

### Backoff Strategy
- Start with the highest-order n-gram (e.g., trigram).
- If the n-gram is **not found**, back off to the next lower-order n-gram (e.g., bigram).
- Continue until the unigram model is reached.
- If no match is found, return the **backoff factor** ($\alpha$).

### Backoff Factor ($\alpha$)
- $\alpha$ is a constant multiplier used for scoring lower-order n-grams.
- It is typically set to $0.4$, as per the original implementation of Stupid Backoff.

---

## 2. Combined Stupid Backoff with Optional Laplace Smoothing

The code combines **Stupid Backoff** with an option for **Laplace Smoothing** to handle zero probabilities.

### Steps:
1. **Start with the Highest n-gram**:
   - Look for the n-gram in the training data.
   - If found:
     - Use Laplace Smoothing:  
       $
       P(w_i|w_{i-(n-1)}, ..., w_{i-1}) = \frac{\text{Count}(w_i, w_{i-(n-1)}, ..., w_{i-1}) + 1}{\text{Count}(w_{i-(n-1)}, ..., w_{i-1}) + V}
       $
       Where $V$ is the vocabulary size.
     - Otherwise, return the relative frequency:
       $
       P(w_i|w_{i-(n-1)}, ..., w_{i-1}) = \frac{\text{Count}(w_i, w_{i-(n-1)}, ..., w_{i-1})}{\text{Count}(w_{i-(n-1)}, ..., w_{i-1})}
       $

2. **Backoff to Lower-Order n-grams**:
   - If the n-gram is not found, reduce the order (e.g., backoff from trigram to bigram).
   - Repeat the probability calculation for the lower-order n-gram.

3. **Fallback to Unigram**:
   - If no n-grams match, compute the unigram probability:
     - With Laplace Smoothing:
       $
       P(w_i) = \frac{\text{Count}(w_i) + 1}{\text{Total Words} + V}
       $
     - Without Laplace Smoothing:
       $
       P(w_i) = \frac{\text{Count}(w_i)}{\text{Total Words}}
       $

4. **Return Backoff Factor**:
   - If the word is entirely unseen, return $\alpha$

In [11]:
def predict_next_word(sequence, ngram_models, alpha=0.4, top_k=5):
    """
    Predict the next word for a given sequence using the Stupid Backoff method.
    Returns:
        list: Top-k predicted words and their scores.
    """
    sequence_tokens = sequence.split()
    candidates = defaultdict(float)

    print(f"\nInput sequence: '{sequence}'")
    print(f"Tokens: {sequence_tokens}")

    # Iterate over all n-grams (from highest available to unigram)
    for n in range(3, 1, -1):  # 3-gram to 1-gram
        ngram_model = ngram_models.get(n)
        if not ngram_model:
            continue
        
        ngram_counts, _, _ = ngram_model
        prefix = tuple(sequence_tokens[-(n - 1):]) if n > 1 else tuple()

        print(f"\nSearching for {n}-grams with prefix: {prefix}")

        for ngram, count in ngram_counts.items():
            if ngram[:-1] == prefix:  # Match the prefix
                word = ngram[-1]
                score = compute_stupid_backoff_probability(ngram, ngram_models, alpha)
                candidates[word] += score
                print(f"Match found: {ngram}, Count: {count}, Score: {score:.6f}")

    # Sort candidates by their scores
    sorted_candidates = sorted(candidates.items(), key=lambda x: x[1], reverse=True)

    print("\nTop candidate words and scores:")
    for word, score in sorted_candidates[:top_k]:
        print(f"Word: {word}, Score: {score:.6f}")

    return sorted_candidates[:top_k]


## Function for correcting the misspelled word

In [20]:
def correct_misspelled_word(word, vocabulary, ngram_models, context, alpha=0.4, top_k=5):
    """
    Predict the correct word for a misspelled word using Stupid Backoff and semantic similarity.
    Returns:
        str: The corrected word.
    """
    # Step 1: Use difflib to get similar words
    similar_words = get_close_matches(word, vocabulary, n=top_k)

    if not similar_words:
        # No similar words found; return the original word
        return word

    # Step 2: Rank similar words using Stupid Backoff
    word_scores = []
    for candidate in similar_words:
        # Add the candidate to the context for scoring
        sequence_tokens = (context.split() + [candidate])
        sequence = " ".join(sequence_tokens)
        prob = sentence_probability_with_backoff(sequence, ngram_models, alpha)
        word_scores.append((candidate, prob))

    # Step 3: Return the word with the highest score
    word_scores.sort(key=lambda x: x[1], reverse=True)
    
    # Step 4: Display the top-k ranked results
    print(f"\nTop-{len(word_scores)} Predictions for '{word}':")
    for rank, (candidate, prob) in enumerate(word_scores, start=1):
        print(f"  {rank}. {candidate} - Probability: {prob:.10e}")  # Scientific notation
    
    return word_scores[0][0] if word_scores else word


## Function for building first 3 n-gram models

In [13]:
def build_ngram_models(filepath):
    """
    Build n-gram models for 1-gram, 2-gram, and 3-gram.

    Args:
        filepath (str): Path to the dataset.

    Returns:
        dict: Dictionary of n-gram models (ngram_counts, n_minus_1_counts, vocab_size) for n = 1, 2, 3.
    """
    corpus = load_data(filepath)
    ngram_models = {}
    
    for n in range(1, 4):  # 1-gram, 2-gram, 3-gram
        ngram_counts, n_minus_1_counts, vocab_size = build_ngram_model(corpus, n)
        ngram_models[n] = (ngram_counts, n_minus_1_counts, vocab_size)
    
    return ngram_models


# Main Function for doing the exercises

## Download tedtalk.txt

In [14]:
# Download tedtalk
url = "https://drive.google.com/file/d/1ZFXJVav0rZ0V2TadMuY0TxWuwxkhN-nq/view?usp=sharing"

def download_from_google_drive(url, output_filename=None):
    # Extract file ID using regex
    match = re.search(r"/d/([^/]+)", url)
    if not match:
        print("Error: Could not extract file ID from the URL.")
        return
    
    file_id = match.group(1)
    print(f"Extracted File ID: {file_id}")

    download_url = f"https://drive.google.com/uc?id={file_id}"

    if output_filename:
        gdown.download(download_url, output_filename, quiet=False)
    else:
        gdown.download(download_url, quiet=False)

url = "https://drive.google.com/file/d/1ZFXJVav0rZ0V2TadMuY0TxWuwxkhN-nq/view?usp=sharing"
download_from_google_drive(url, "tedtalk.txt")


Extracted File ID: 1ZFXJVav0rZ0V2TadMuY0TxWuwxkhN-nq


Downloading...
From: https://drive.google.com/uc?id=1ZFXJVav0rZ0V2TadMuY0TxWuwxkhN-nq
To: e:\2_LEARNING_BKU\2_File_2\K22_HK242\CO3085_NLP\BT\Lab03\tedtalk.txt
100%|██████████| 40.3M/40.3M [00:04<00:00, 9.09MB/s]


## Compare with the results from In Class Exercise.

In [15]:
dataset_path = os.path.join(os.getcwd(), "tedtalk.txt")

print("Building n-gram models...")
ngram_models = build_ngram_models(dataset_path)

# Example sentences for comparison
correct_sentence = "the cat sat on the mat"
incorrect_sentence = "the mat sat on the cat"

# Compare results
comparison_results = compare_sentences_with_smoothing(correct_sentence, incorrect_sentence, ngram_models)
print("\nComparison Results:")
for method, results in comparison_results.items():
    print(f"\n{method} Smoothing:")
    if method == "Laplace":
        # Loop each n-gram
        for ngram, probs in results.items():
            print(f"{ngram} - Correct: {probs['sentence1']}, Incorrect: {probs['sentence2']}")
            print(f"  Higher Probability: {probs['higher_probability']}")
            print(f"  Probability Difference: {probs['probability_difference']}")
    elif method == "Stupid Backoff":
        # Print result for Stupid Backoff
        print(f"Correct: {results['sentence1']}, Incorrect: {results['sentence2']}")
        print(f"  Higher Probability: {results['higher_probability']}")
        print(f"  Probability Difference: {results['probability_difference']}")


Building n-gram models...

Comparison Results:

Laplace Smoothing:
1-gram - Correct: 5.360670754385347e-27, Incorrect: 5.360670754385346e-27
  Higher Probability: Sentence 1 (Correct)
  Probability Difference: 7.174648137343064e-43
2-gram - Correct: 9.04981020727541e-29, Incorrect: 9.049810207275411e-29
  Higher Probability: Sentence 2 (Incorrect)
  Probability Difference: 1.1210387714598537e-44
3-gram - Correct: 4.7232926503721375e-31, Incorrect: 4.723292650372137e-31
  Higher Probability: Sentence 1 (Correct)
  Probability Difference: 8.758115402030107e-47

Stupid Backoff Smoothing:
Correct: 1.456742136608319e-39, Incorrect: 1.4553198356546868e-39
  Higher Probability: Sentence 1 (Correct)
  Probability Difference: 1.4223009536322667e-42


## Use the newly built model to generate the next words for a given word sequence.

In [16]:
# Predict the next word
sequence = "the cat"

# Predict next words
# I only search from 2 and 3-gram models as 1-gram contain less information
predicted_words = predict_next_word(sequence, ngram_models, alpha=0.4, top_k=5)

print(f"\nTop predictions for '{sequence}':")
for rank, (word, score) in enumerate(predicted_words, start=1):
    print(f"{rank}. {word} - Score: {score:.6f}")



Input sequence: 'the cat'
Tokens: ['the', 'cat']

Searching for 3-grams with prefix: ('the', 'cat')
Match found: ('the', 'cat', 'sleep'), Count: 1, Score: 0.000011
Match found: ('the', 'cat', 'likes'), Count: 1, Score: 0.000011
Match found: ('the', 'cat', 'came'), Count: 1, Score: 0.000011
Match found: ('the', 'cat', 'scan.'), Count: 1, Score: 0.000011
Match found: ('the', 'cat', 'scan'), Count: 2, Score: 0.000017
Match found: ('the', 'cat', 'is'), Count: 8, Score: 0.000051
Match found: ('the', 'cat', 'scans,'), Count: 1, Score: 0.000011
Match found: ('the', 'cat', 'from'), Count: 1, Score: 0.000011
Match found: ('the', 'cat', 'can'), Count: 1, Score: 0.000011
Match found: ('the', 'cat', 'and'), Count: 2, Score: 0.000017
Match found: ('the', 'cat', 'to'), Count: 1, Score: 0.000011
Match found: ('the', 'cat', 'scanner,'), Count: 1, Score: 0.000011
Match found: ('the', 'cat', 'eats'), Count: 1, Score: 0.000011
Match found: ('the', 'cat', 'or'), Count: 1, Score: 0.000011
Match found: ('t

## Combine with a function that calculates the distance between words to predict the correct word for a misspelled word position. (from difflib import get_close_matches)

In [21]:
# Example vocabulary and models
vocabulary = set(load_data("tedtalk.txt"))

# Example misspelled word correction
misspelled_word = "wonderfall"
context = "I love this"
corrected_word = correct_misspelled_word(misspelled_word, vocabulary, ngram_models, context, alpha=0.4, top_k=5)

print(f"Misspelled Word: {misspelled_word}")
print(f"Corrected Word: {corrected_word}")

misspelled_word = "hhellp"
context = "Thankyou very much for your"
corrected_word = correct_misspelled_word(misspelled_word, vocabulary, ngram_models, context, alpha=0.4, top_k=5)

print(f"Misspelled Word: {misspelled_word}")
print(f"Corrected Word: {corrected_word}")


Top-5 Predictions for 'wonderfall':
  1. wonderful - Probability: 6.5913879405e-21
  2. wonderfully - Probability: 2.9877586682e-21
  3. wonderfully. - Probability: 4.7175136867e-22
  4. wonderful? - Probability: 4.7175136867e-22
  5. wonderfully, - Probability: 1.5725045622e-22
Misspelled Word: wonderfall
Corrected Word: wonderful

Top-5 Predictions for 'hhellp':
  1. helps - Probability: 8.9527142795e-28
  2. hell - Probability: 4.2397215200e-28
  3. help - Probability: 2.4737207895e-28
  4. shell - Probability: 1.5578511632e-28
  5. he’ll - Probability: 2.3663561972e-29
Misspelled Word: hhellp
Corrected Word: helps
